In [0]:
# Importing needed dependencies
from pyspark.sql import *
import pyspark.sql.functions as F
from pyspark.sql.functions import to_json, col
import json as j

from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import from_json, col, when
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.functions import col, to_timestamp, to_date

In [0]:
# Listing the files in the bronze folder
dbutils.fs.ls('mnt/bronze/')

[FileInfo(path='dbfs:/mnt/bronze/business.json', name='business.json', size=3498714, modificationTime=1749577662000),
 FileInfo(path='dbfs:/mnt/bronze/categories.json', name='categories.json', size=11560, modificationTime=1749577711000),
 FileInfo(path='dbfs:/mnt/bronze/review.json', name='review.json', size=338843, modificationTime=1749577787000),
 FileInfo(path='dbfs:/mnt/bronze/sub_categories.json', name='sub_categories.json', size=84165, modificationTime=1749577753000),
 FileInfo(path='dbfs:/mnt/bronze/user.json', name='user.json', size=878114, modificationTime=1749577819000)]

# Transforming the Json files into Tables

In [0]:
# Load the business JSON file
df1 = spark.read.json("dbfs:/mnt/bronze/business.json")
df1.printSchema()

root
 |-- __v: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- address: string (nullable = true)
 |-- addressVerified: boolean (nullable = true)
 |-- bookmarks: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- branchesInOtherLocations: boolean (nullable = true)
 |-- businessCategory: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- businessDescription: string (nullable = true)
 |-- businessEmail: string (nullable = true)
 |-- businessEmailVerified: boolean (nullable = true)
 |-- businessMedia: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- businessName: string (nullable = true)
 |-- businessPhoneNumber: string (nullable = true)
 |-- businessPhoneNumberVerified: boolean (nullable = true)
 |-- businessProductAndService: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- businessSu

In [0]:
oid_schema = StructType([
    StructField("$oid", StringType(), True)
])

In [0]:
df_business = df1.withColumn('business_category', F.explode('businessCategory')).\
                withColumn('business_subcategory', F.explode('businessSubCategory')).\
                withColumn('business_id', F.col('_id.$oid')).\
                withColumn(
                        "business_categories_id",
                        regexp_replace(col("business_category"), r'^\{"\$oid":"|"\}$', '')
                    ).\
                withColumn(
                        "business_subcategories_id",
                        regexp_replace(col("business_subcategory"), r'^\{"\$oid":"|"\}$', '')
                    ).\
                select('business_id','address','business_categories_id','business_subcategories_id','businessName',
                'businessPhoneNumber','city','state','claimed','contactPerson',
                'businessEmail')

display(df_business)




business_id,address,business_categories_id,business_subcategories_id,businessName,businessPhoneNumber,city,state,claimed,contactPerson,businessEmail
65eac2edea78b88f1875bf55,"Apatapiti Rd, Akure 340110, Ondo, Nigeria",65e05b0fd460100f731eb020,65e05b0fd460100f731eb022,AnimeCon,+2349035608856,Akure,Ondo,false,Victor Adekunle,victoradekunle312@gmail.com
65eac2edea78b88f1875bf55,"Apatapiti Rd, Akure 340110, Ondo, Nigeria",65e05b0fd460100f731eb020,65e05b10d460100f731eb026,AnimeCon,+2349035608856,Akure,Ondo,false,Victor Adekunle,victoradekunle312@gmail.com
65ead2abea78b88f1875c118,,65e05b13d460100f731eb038,65e05b13d460100f731eb03a,Chicken,2347031090186,Akure,Ondo,false,,cr4lan952@mozmail.com
65ead354ea78b88f1875c14f,,65e05b4cd460100f731eb196,65e05b4dd460100f731eb19a,Odopos Properties Limited,2349065153811,Akure,Ondo State,false,Mr John,odoposltd@yahoo.com
65ead354ea78b88f1875c14f,,65e05b4cd460100f731eb196,65e05b4dd460100f731eb198,Odopos Properties Limited,2349065153811,Akure,Ondo State,false,Mr John,odoposltd@yahoo.com
65ead354ea78b88f1875c14f,,65e05b4cd460100f731eb196,65e05b4ed460100f731eb19c,Odopos Properties Limited,2349065153811,Akure,Ondo State,false,Mr John,odoposltd@yahoo.com
65ead371ea78b88f1875c174,,65e05b45d460100f731eb168,65e05b45d460100f731eb16a,Groovy Hotel And Suites,2348102774291,Akure,Ondo,false,,info@groovyhotelclub.com
65ead406ea78b88f1875c1f0,,65e05b00d460100f731eafb9,65e05b00d460100f731eafbc,Angel Florah,2348149314437,Akure,Ondo,false,,angelflorah01@gmail.com
65ead406ea78b88f1875c1f0,,65e05b00d460100f731eafb9,65e05b02d460100f731eafd0,Angel Florah,2348149314437,Akure,Ondo,false,,angelflorah01@gmail.com
65ead406ea78b88f1875c1f0,,65e05b00d460100f731eafb9,65e05b02d460100f731eafc8,Angel Florah,2348149314437,Akure,Ondo,false,,angelflorah01@gmail.com


In [0]:
### to save the business file in a paruet file

df_business.write.mode('overwrite').parquet("dbfs:/mnt/silver/business")

In [0]:
# Load the category JSON file
df2 = spark.read.json("dbfs:/mnt/bronze/categories.json")
df2.printSchema()

root
 |-- __v: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- createdAt: struct (nullable = true)
 |    |-- $date: struct (nullable = true)
 |    |    |-- $numberLong: string (nullable = true)
 |-- imageUrl: string (nullable = true)
 |-- metrics: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: string (nullable = true)
 |-- slug: string (nullable = true)
 |-- status: string (nullable = true)



In [0]:
df_categories = df2.withColumn('metrics', F.explode('metrics')).\
                withColumn('categories_id', F.col('_id.$oid')).\
                 select('categories_id','metrics','name','slug','status')
display(df_categories)

categories_id,metrics,name,slug,status
65e05afed460100f731eaf9e,Food Quality,Restaurants And Bars,restaurants-and-bars,Active
65e05afed460100f731eaf9e,Customer Service,Restaurants And Bars,restaurants-and-bars,Active
65e05afed460100f731eaf9e,Wait Time,Restaurants And Bars,restaurants-and-bars,Active
65e05afed460100f731eaf9e,Cleanliness & Ambience,Restaurants And Bars,restaurants-and-bars,Active
65e05b00d460100f731eafb9,Product Quality,Fashion & Apparel,fashion-and-apparel,Active
65e05b00d460100f731eafb9,Customer Service,Fashion & Apparel,fashion-and-apparel,Active
65e05b00d460100f731eafb9,Packaging,Fashion & Apparel,fashion-and-apparel,Active
65e05b00d460100f731eafb9,Timeliness,Fashion & Apparel,fashion-and-apparel,Active
65e05b03d460100f731eafd4,Quality of Service,Home Services,home-services,Active
65e05b03d460100f731eafd4,Customer Service,Home Services,home-services,Active


In [0]:
### to save the categories file in a parquet file

df_categories.write.mode('overwrite').parquet("dbfs:/mnt/silver/categories")

In [0]:
# Load the sub_category JSON file
df3 = spark.read.json("dbfs:/mnt/bronze/sub_categories.json")
df3.printSchema()

root
 |-- __v: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- categoryIds: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- $oid: string (nullable = true)
 |-- createdAt: struct (nullable = true)
 |    |-- $date: struct (nullable = true)
 |    |    |-- $numberLong: string (nullable = true)
 |-- description: string (nullable = true)
 |-- name: string (nullable = true)
 |-- slug: string (nullable = true)
 |-- status: string (nullable = true)
 |-- updatedAt: struct (nullable = true)
 |    |-- $date: struct (nullable = true)
 |    |    |-- $numberLong: string (nullable = true)



In [0]:
df_subcategories = df3.withColumn('categories_id', F.explode('categoryIds')).\
                   withColumn('subcategories_id', F.col('_id.$oid')).\
                   withColumn('categories_id', F.col('categories_id.$oid')).\
                 select('subcategories_id','categories_id','name','slug','status','description')
display(df_subcategories)

subcategories_id,categories_id,name,slug,status,description
65e05afed460100f731eafa2,65e05afed460100f731eaf9e,African and Pacific Cuisine,african-and-pacific-cuisine,Active,Traditional African and Pacific cuisine.
65e05afed460100f731eafa4,65e05afed460100f731eaf9e,Bars and Cafes,bars-and-cafes,Active,Bars and cafes for food and drinks.
65e05afed460100f731eafa6,65e05afed460100f731eaf9e,Nigerian Cuisine,nigerian-cuisine,Active,Service establishments offering fast meals.
65e05affd460100f731eafa9,65e05afed460100f731eaf9e,European Cuisine,european-cuisine,Active,European cuisine for food and drinks.
65e05affd460100f731eafab,65e05afed460100f731eaf9e,General Restaurants,general-restaurants,Active,General restaurants for food and drinks.
65e05affd460100f731eafae,65e05afed460100f731eaf9e,Launch,launch,Active,Launch for food and drinks.
65e05affd460100f731eafb1,65e05afed460100f731eaf9e,Dinner,dinner,Active,Dinner for food and drinks.
65e05b00d460100f731eafb4,65e05afed460100f731eaf9e,Breakfast,breakfast,Active,Breakfast for food and drinks.
65e05b00d460100f731eafb7,65e05afed460100f731eaf9e,Snacks,snacks,Active,Snacks for food and drinks.
65e05b00d460100f731eafbc,65e05b00d460100f731eafb9,Accessories,accessories,Active,Complementary items to enhance fashion outfits.


In [0]:
### to save the sub_categories file in a parquet file

df_subcategories.write.mode('overwrite').parquet("dbfs:/mnt/silver/subcategories")

In [0]:
# Load the users JSON file
df4 = spark.read.json("dbfs:/mnt/bronze/user.json")
df4.printSchema()

root
 |-- __v: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- bookmarks: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- $oid: string (nullable = true)
 |-- bvn: string (nullable = true)
 |-- city: string (nullable = true)
 |-- createdAt: struct (nullable = true)
 |    |-- $date: struct (nullable = true)
 |    |    |-- $numberLong: string (nullable = true)
 |-- dateOfBirth: struct (nullable = true)
 |    |-- $date: struct (nullable = true)
 |    |    |-- $numberLong: string (nullable = true)
 |-- deviceTokens: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _id: struct (nullable = true)
 |    |    |    |-- $oid: string (nullable = true)
 |    |    |-- platform: string (nullable = true)
 |    |    |-- token: string (nullable = true)
 |-- email: string (nullable = true)
 |-- emailVerified: boolean (nullable

In [0]:
df_users = df4.withColumn('users_id', F.col('_id.$oid')).\
            select('users_id','firstName','lastName','gender','state','username','city','email','verificationStatus')
display(df_users)

users_id,firstName,lastName,gender,state,username,city,email,verificationStatus
66e9c426e970e655731d0be8,Adekola,Owoade,Male,Ondo,Petra,Akure,owoadekola95@gmail.com,true
66ea9090e970e655731d0cef,Seyi,Oke,null,,Ragna,,appmeseyi6@gmail.com,false
66eac1270389b10cf8647f95,Daniel,Oyekunle,null,,dcares,,dannicares@gmail.com,false
66eaedba0389b10cf8648040,Dolapo,Adedoyin,Female,Ondo,Dola,Akure,dholliepearl5@gmail.com,true
66eaef080389b10cf8648081,Reese,Uvoh,Female,Ondo,Marigold,Akure,uvohprecious06@gmail.com,true
66eaef1b0389b10cf8648095,Boluwatife,Oludupin,Male,Ondo,Bovico,Akure,bolu.oludupin@gmail.com,true
66eaef260389b10cf86480a8,Ridwan,Mutairu,null,,Riidos,,mutairuridwan22@gmail.com,false
66eaf0190389b10cf8648153,Olumide,Olowu,null,Ondo,olumide,Akure,olumideayool17@gmail.com,false
66eaf13c0389b10cf864821d,Samuel,Olaniyi,Male,Ondo,Samuel,Akure,olaniyidsam@gmail.com,true
66eaf16e0389b10cf864824d,Hellen,Ogunmola,Female,Ondo,HelMart,Akure,hellenoguns96@gmail.com,true


In [0]:
### to save the users file in a parquet file

df_users.write.mode('overwrite').parquet("dbfs:/mnt/silver/users")

In [0]:
# Load the reviews JSON file
df5 = spark.read.json("dbfs:/mnt/bronze/review.json")
df5.printSchema()

root
 |-- __v: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- bookmarkedBy: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- $oid: string (nullable = true)
 |-- business: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- businessCategory: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- createdAt: struct (nullable = true)
 |    |-- $date: struct (nullable = true)
 |    |    |-- $numberLong: string (nullable = true)
 |-- dateOfExperience: struct (nullable = true)
 |    |-- $date: struct (nullable = true)
 |    |    |-- $numberLong: string (nullable = true)
 |-- downVoters: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- $oid: string (nullable = true)
 |-- downVotes: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- experienceSummary: stri

In [0]:
# to define the date schema
numberLong_schema = StructType([
    StructField("$numberLong", StringType(), True)
])

In [0]:
df_reviews = df5.withColumn('reviews_id', F.col('_id.$oid')).\
                withColumn('business_id', F.col('business.$oid')).\
                withColumn('user', F.col('user.$oid')).\
                withColumn("business_category_id", F.col("businessCategory.$oid")).\
                withColumn(
                                "dateOfExperience",
                                to_timestamp((col("dateOfExperience.$date.$numberLong").cast("long") / 1000))
                            ).\
                withColumn(
                                "date_of_experience",
                                to_date("dateOfExperience")
                            ).\
                    withColumn('Accessibility', col('rating.Accessibility.$numberInt').cast("int")).\
                    withColumn('Appointment Flexibility', col('rating.Appointment Flexibility.$numberInt').cast("int")).\
                    withColumn('Buying Experience', col('rating.Buying Experience.$numberInt').cast("int")).\
                    withColumn('Classes and Programs', col('rating.Classes & Programs.$numberInt').cast("int")).\
                    withColumn('Cleanliness and Ambience', col('rating.Cleanliness & Ambience.$numberInt').cast("int")).\
                    withColumn('Customer Service', col('rating.Customer Service.$numberInt').cast("int")).\
                    withColumn('Equipment Quality', col('rating.Equipment Quality.$numberInt').cast("int")).\
                    withColumn('Food and Dining', col('rating.Food & Dining.$numberInt').cast("int")).\
                    withColumn('Food Quality', col('rating.Food Quality.$numberInt').cast("int")).\
                    withColumn('Learning Experience', col('rating.Learning Experience.$numberInt').cast("int")).\
                    withColumn('Networking and Interaction', col('rating.Networking & Interaction.$numberInt').cast("int")).\
                    withColumn('Organization', col('rating.Organization.$numberInt').cast("int")).\
                    withColumn('Product Quality', col('rating.Product Quality.$numberInt').cast("int")).\
                    withColumn('Professionalism', col('rating.Professionalism.$numberInt').cast("int")).\
                    withColumn('Quality of Care', col('rating.Quality of Care.$numberInt').cast("int")).\
                    withColumn('Quality of Curriculum', col('rating.Quality of Curriculum.$numberInt').cast("int")).\
                    withColumn('Quality of Instructors', col('rating.Quality of Instructors.$numberInt').cast("int")).\
                    withColumn('Quality of Service', col('rating.Quality of Service.$numberInt').cast("int")).\
                    withColumn('Room Quality', col('rating.Room Quality.$numberInt').cast("int")).\
                    withColumn('Safety and Security', col('rating.Safety & Security.$numberInt').cast("int")).\
                    withColumn('Security', col('rating.Security.$numberInt').cast("int")).\
                    withColumn('Service Quality', col('rating.Service Quality.$numberInt').cast("int")).\
                    withColumn('Session Quality', col('rating.Session Quality.$numberInt').cast("int")).\
                    withColumn('Timeliness', col('rating.Timeliness.$numberInt').cast("int")).\
                    withColumn('Wait Time', col('rating.Wait Time.$numberInt').cast("int")).\
                        select("reviews_id","business_id","business_category_id","date_of_experience",
                        F.col("experienceSummary").alias("reviews"),
                        F.col("user").alias("user_id"),
                        F.col("Accessibility"),
                        F.col("Appointment Flexibility"),
                        F.col("Buying Experience"),
                        F.col("Classes and Programs"),
                        F.col("Cleanliness and Ambience"),
                        F.col("Customer Service"),
                        F.col("Equipment Quality"),
                        F.col("Food and Dining"),
                        F.col("Food Quality"),
                        F.col("Learning Experience"),
                        F.col("Networking and Interaction"),
                        F.col("Organization"),
                        F.col("Product Quality"),
                        F.col("Professionalism"),
                        F.col("Quality of Care"),
                        F.col("Quality of Curriculum"),
                        F.col("Quality of Instructors"),
                        F.col("Quality of Service"),
                        F.col("Room Quality"),
                        F.col("Safety and Security"),
                        F.col("Security"),
                        F.col("Service Quality"),
                        F.col("Session Quality"),
                        F.col("Timeliness"),
                        F.col("Wait Time")                    )




display(df_reviews)

reviews_id,business_id,business_category_id,date_of_experience,reviews,user_id,Accessibility,Appointment Flexibility,Buying Experience,Classes and Programs,Cleanliness and Ambience,Customer Service,Equipment Quality,Food and Dining,Food Quality,Learning Experience,Networking and Interaction,Organization,Product Quality,Professionalism,Quality of Care,Quality of Curriculum,Quality of Instructors,Quality of Service,Room Quality,Safety and Security,Security,Service Quality,Session Quality,Timeliness,Wait Time
66e97ad9750d40d85cd6b797,65f306614e5b5b9c84a86846,65e05afed460100f731eaf9e,2024-09-17,"Honestly, Ovie makes really good food. Her prices are way too exorbitant but her food tastes real good. But is it worth it imo? I really don't think so. I mean how will a spoon of rice be 700??? Her spot too isn't nice she needs to move a better location if she'll keep collect such amount. What do you guys think?",66e978c5750d40d85cd6b726,null,null,null,null,3,3,null,null,4,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,3
66eaf0cb0389b10cf8648195,66e187dd1d1ca70126cc0b56,65e05b25d460100f731eb0b0,2024-09-18,"Honestly, ZACRAC learning is one of the best learning businesses I’ve patronized.",66ea931de970e655731d0d50,null,null,null,null,null,4,null,null,null,5,null,null,null,null,null,5,5,null,null,null,null,null,null,null,null
66eaf0dd0389b10cf86481c4,65f3008d4e5b5b9c84a86308,65e05afed460100f731eaf9e,2024-09-18,"I recently ordered some food at Ifeoma, and it was an exceptional experience from start to finish! The ambiance is warm and welcoming, with a cozy yet elegant vibe that makes you feel right at home. The staff is incredibly friendly and attentive, ensuring that every detail of our visit was perfect. Now, let’s talk about the food – absolutely delicious! The menu offers a great mix of traditional and modern dishes, with bold flavors and fresh ingredients. I highly recommend trying the chef’s specials; every bite was a burst of flavor, and the presentation was beautiful. Ifeoma is truly a gem, and I can’t wait to return. Whether you're dining with friends, family, or on a date, this restaurant is a must-visit. Highly recommended!",66e9c426e970e655731d0be8,null,null,null,null,5,5,null,null,5,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,5
66eaf27f0389b10cf86482e0,65f306614e5b5b9c84a86846,65e05afed460100f731eaf9e,2024-09-12,"I really like the food there, and it was heated to make sure I don't eat cold food, but the food is quite pricey and and I don't like that it's outside",66eaf0190389b10cf8648153,null,null,null,null,3,4,null,null,4,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,5
66eeb8ea15b0efb41b861968,66e187dd1d1ca70126cc0b56,65e05b25d460100f731eb0b0,2024-09-21,They are great at what they do. Although their fee is higher than others but you will get more value than the fee. Thanks Zacrac Learning,66eeb7ef15b0efb41b86193d,null,null,null,null,null,4,null,null,null,5,null,null,null,null,null,5,5,null,null,null,null,null,null,null,null
66eeb9d615b0efb41b861990,662b898edfc48ac4557deb8c,65e05b1ad460100f731eb06c,2024-09-21,"My session was great , and the dentists were also friendly. Although I felt some pain during the session. But overall, great experience. A bit pricey",66eeb7ef15b0efb41b86193d,null,5,null,null,null,3,null,null,null,null,null,null,null,null,5,null,null,null,null,null,null,null,null,null,5
66efb33915b0efb41b861c9b,65f306614e5b5b9c84a86846,65e05afed460100f731eaf9e,2024-09-22,"Well, whenever I need to eat a food that tastes like homemade food, Ovie is my go to person though her food is a bit expensive but it's understandable considering the prices of things in the country. However I think she needs to get a proper space as eating in an open place isn't really cool.",66ec4c0915b0efb41b861790,null,null,null,null,3,2,null,null,4,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2
66efb49e15b0efb41b861d29,66e187dd1d1ca70

In [0]:
### to save the reviews file in a parquet file

df_reviews.write.mode('overwrite').parquet("dbfs:/mnt/silver/reviews")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-7283909769339479>, line 1
----> 1 df_reviews = df5.withColumn('_id', F.col('_id.$oid')).\
      2                 withColumn('business_id', F.col('business.$oid')).\
      3                 withColumn('user', F.col('user.$oid')).\
      4                 withColumn("businessCategory", F.col("businessCategory.$oid")).\
      5                 withColumn(
      6                                 "dateOfExperience",
      7                                 to_timestamp((col("dateOfExperience.$date.$numberLong").cast("long") / 1000))
      8                             ).\
      9                 withColumn(
     10                                 "dateOfExperience",
     11                                 to_date("dateOfExperience")
     12                             ).\
     13                     withColumn('Accessibility', 